# Identify settlements

This notebook identifies settlements that can be used as origins in an accessibility analysis for Khbyer Pakhtunkhwa. Several different identification criteria are explored

In [53]:
import os, sys, time

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape
from shapely.wkt import loads
from shapely.errors import TopologicalError

import rasterio
from rasterio import features, transform
from rasterio.mask import mask
from rasterio.transform import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling

import numpy as np

from rasterstats import zonal_stats

sys.path.append('../../src/')

import skimage.graph as graph
import gostrocks.src.GOSTRocks as gr
from gostrocks.src.GOSTRocks.misc import tPrint
from GOSTNets_Raster.src.GOSTNets_Raster import market_access as ma

File paths

In [54]:
geo_pth = r"/Users/robert/Documents/Jobs/WB/Pakistan/Data"
pop_pth = r"/Users/robert/Documents/Jobs/WB/Pakistan/Data/Population"

In [55]:
data_dir = r'../../data'
vect_dir = r'vect_inputs'
rast_dir = r'rast_inputs'
out_pth = r'../../data/vect_out'

Projections

In [56]:
dest_crs = 'EPSG:32642' # this is a Pakistani UTM projection, assign correct projection for project area

Functions

In [57]:
# Lightly adapted from https://gis.stackexchange.com/questions/290030/what-does-it-mean-to-reproject-a-satellite-image-from-utm-zone-13n-to-wgs84

def reproject_tif(source_file, destination_file,dest_crs):
    """Re-projects tif at source file to destination CRS at destination file.

    Args:
        source_file: file to re-project
        destination_file: file to store re-projection

    Returns:
        destination_file: where the re-projected file is saved at
    """

    with rasterio.open(source_file) as src:
        dst_crs = dest_crs
        transform, width, height = calculate_default_transform(
            src.crs,
            dst_crs,
            src.width,
            src.height,
            *src.bounds
        )

        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height,
            "compress":'LZW'
        })

        with rasterio.open(destination_file, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest,
                    num_threads=-1
                )

        return destination_file

## Load in raster data

WSF Files

In [58]:
wsf_raw_fil = 'KP_WSF2019_binary.tif'
wsf_3x3_fil = 'KP_WSF2019_binary_skimage_modal_3by3.tif'
wsf_5x5_fil = 'KP_WSF2019_binary_skimage_modal_5by5.tif'
wsf_7x7_fil = 'KP_WSF2019_binary_skimage_modal_7by7.tif'

In [59]:
reproject_tif(os.path.join(data_dir,rast_dir,wsf_raw_fil),
             os.path.join(data_dir,rast_dir,'KP_WSF2019_binary_32642.tif'),
             'EPSG:32642')

'../../data/rast_inputs/KP_WSF2019_binary_32642.tif'

In [60]:
with rasterio.open(os.path.join(data_dir,rast_dir,'KP_WSF2019_binary_32642.tif'),'r') as wsf_raw_raw:
    wsf_raw = wsf_raw_raw.read(1)
    wsf_raw_prof = wsf_raw_raw.profile

Now WorldPop

In [61]:
wp_kp_fil = 'Population/WorldPop/KP_Analysis_WorldPop_2020.tif'

In [62]:
with rasterio.open(os.path.join(geo_pth,wp_kp_fil),'r') as wp_kp_raw:
    wp_kp = wp_kp_raw.read(1)
    wk_kp_prof = wp_kp_raw.profile

#### Settlements

In [63]:
settle = gpd.read_file(os.path.join(data_dir,vect_dir,'KP_Settlements_NGA_2017.gpkg'),driver="GPKG")
settle = settle.set_crs(4326).to_crs(dest_crs)

/Users/robert/.pyenv/versions/geo/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [64]:
# GeoNames contains duplicate values for places with alternate names (curiously). We need to drop them
settle = settle[settle.NT == 'N']

In [65]:
filter_list = ['PPL', 'PPLA', 'PPLA2']

In [66]:
settle = settle[settle['DSG'].isin(filter_list) == True]

In [67]:
len(settle)

24625

In [68]:
settle = settle.reset_index().drop('index',axis=1)

In [69]:
settle.head()

,FID_Settle,RC,UFI,UNI,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,...,FID_Pak_ad,PROVINCE,PROVINCE_C,DISTRICT,DISTRICT_C,TEHSIL,TEHSIL_C,Remarks,hfs,geometry
0,105912,5,6048637.0,15681640.0,31.304456,70.340982,31:18:16N,70:20:28E,42RXV2761564120,NH42-03,...,267,Khyber Pakhtunkhwa,4.0,FR Dera Ismail Khan,409,FR Dera Ismail Khan,40901.0,Pcode change,0,POINT (627740.770 3464116.408)
1,26327,5,6143383.0,6151664.0,31.318889,70.775000,31:19:08N,70:46:30E,42RXV6889866304,NH42-04,...,335,Khyber Pakhtunkhwa,2.0,Dera Ismail Khan,207,Dera Ismail Khan,20702.0,None,0,POINT (668898.353 3466306.068)
2,44285,5,6352950.0,6574703.0,31.327376,70.598907,31:19:39N,70:35:56E,42RXV5212766988,NH42-04,...,335,Khyber Pakhtunkhwa,2.0,Dera Ismail Khan,207,Dera Ismail Khan,20702.0,None,0,POINT (652125.948 3466991.740)
3,44274,5,6140344.0,6574679.0,31.330898,70.529831,31:19:51N,70:31:47E,42RXV4554867285,NH42-04,...,333,Khyber Pakhtunkhwa,2.0,Dera Ismail Khan,207,Daraban,20701.0,None,0,POINT (645545.368 3467286.350)
4,44188,5,6140332.0,6574666.0,31.331360,70.698876,31:19:53N,70:41:56E,42RXV6163267572,NH42-04,...,140,Punjab,6.0,Dera Ghazi Khan,607,Taunsa,60703.0,None,0,POINT (661500.357 3467653.119)


Transform settlements

In [70]:
set200buff = settle.copy()
set200buff.geometry = set200buff.geometry.buffer(200)

In [71]:
set200buff.to_file(os.path.join(data_dir,vect_dir,"KP_Settlements_NGA_2017_200mBuff_32642.shp"))

In [72]:
settle.head(2)

,FID_Settle,RC,UFI,UNI,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,...,FID_Pak_ad,PROVINCE,PROVINCE_C,DISTRICT,DISTRICT_C,TEHSIL,TEHSIL_C,Remarks,hfs,geometry
0,105912,5,6048637.0,15681640.0,31.304456,70.340982,31:18:16N,70:20:28E,42RXV2761564120,NH42-03,...,267,Khyber Pakhtunkhwa,4.0,FR Dera Ismail Khan,409,FR Dera Ismail Khan,40901.0,Pcode change,0,POINT (627740.770 3464116.408)
1,26327,5,6143383.0,6151664.0,31.318889,70.775000,31:19:08N,70:46:30E,42RXV6889866304,NH42-04,...,335,Khyber Pakhtunkhwa,2.0,Dera Ismail Khan,207,Dera Ismail Khan,20702.0,None,0,POINT (668898.353 3466306.068)


## Aggregate WSF footprints within settlements

In [73]:
# run zonal stats on just the geometry column of set200, then join it back to the original GDF based on common indices

set200_zs = set200buff.join(
    pd.DataFrame(
        zonal_stats(
                 vectors = set200buff[['geometry','FID_Settle']],\
                 raster = wsf_raw,\
                 affine = wsf_raw_raw.transform,\
                 stats = 'sum'
        )
    ),
    how='left',on='FID_Settle'
)

/Users/robert/.pyenv/versions/geo/lib/python3.9/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [74]:
# run zonal stats on just the geometry column of set200, then join it back to the original GDF based on common indices

settle_pts_zs = settle.join(
    pd.DataFrame(
        zonal_stats(
                 vectors = set200buff['geometry'],\
                 raster = wsf_raw,\
                 affine = wsf_raw_raw.transform,\
                 stats = 'sum'
        )
    ),
    how='left'
)

In [75]:
set200_zs.tail()

,FID_Settle,RC,UFI,UNI,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,...,PROVINCE,PROVINCE_C,DISTRICT,DISTRICT_C,TEHSIL,TEHSIL_C,Remarks,hfs,geometry,sum
24620,141210,5,13547314.0,17144204.0,36.863020,73.469833,36:51:47N,73:28:11E,43SCA6360380770,NJ43-13,...,Khyber Pakhtunkhwa,2.0,Chitral,206,Mastuj,20602.0,None,0,"POLYGON ((898731.806 4089012.209, 898730.843 4...",NaN
24621,142643,5,10782804.0,17147188.0,36.864675,73.618352,36:51:53N,73:37:06E,43SCA7684680751,NJ43-14,...,Khyber Pakhtunkhwa,2.0,Chitral,206,Mastuj,20602.0,None,0,"POLYGON ((911979.930 4089833.057, 911978.967 4...",NaN
24622,141213,5,13547317.0,17144207.0,36.864831,73.398221,36:51:53N,73:23:54E,43SCA5722381075,NJ43-13,...,Khyber Pakhtunkhwa,2.0,Chitral,206,Mastuj,20602.0,None,0,"POLYGON ((892335.098 4088914.911, 892334.135 4...",NaN
24623,150286,5,10782813.0,11604437.0,36.868793,73.377642,36:52:08N,73:22:40E,43SCA5539681546,NJ43-13,...,Khyber Pakhtunkhwa,2.0,Chitral,206,Mastuj,20602.0,None,0,"POLYGON ((890477.037 4089274.401, 890476.074 4...",NaN
24624,135201,5,10782803.0,11604427.0,36.874666,73.651880,36:52:29N,73:39:07E,43SCA7985081817,NJ43-14,...,Khyber Pakhtunkhwa,2.0,Chitral,206,Mastuj,20602.0,None,0,"POLYGON ((914914.445 4091088.767, 914913.482 4...",NaN


In [76]:
settle_pts_zs.tail()

,FID_Settle,RC,UFI,UNI,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,...,PROVINCE,PROVINCE_C,DISTRICT,DISTRICT_C,TEHSIL,TEHSIL_C,Remarks,hfs,geometry,sum
24620,141210,5,13547314.0,17144204.0,36.863020,73.469833,36:51:47N,73:28:11E,43SCA6360380770,NJ43-13,...,Khyber Pakhtunkhwa,2.0,Chitral,206,Mastuj,20602.0,None,0,POINT (898531.806 4089012.209),15.0
24621,142643,5,10782804.0,17147188.0,36.864675,73.618352,36:51:53N,73:37:06E,43SCA7684680751,NJ43-14,...,Khyber Pakhtunkhwa,2.0,Chitral,206,Mastuj,20602.0,None,0,POINT (911779.930 4089833.057),0.0
24622,141213,5,13547317.0,17144207.0,36.864831,73.398221,36:51:53N,73:23:54E,43SCA5722381075,NJ43-13,...,Khyber Pakhtunkhwa,2.0,Chitral,206,Mastuj,20602.0,None,0,POINT (892135.098 4088914.911),0.0
24623,150286,5,10782813.0,11604437.0,36.868793,73.377642,36:52:08N,73:22:40E,43SCA5539681546,NJ43-13,...,Khyber Pakhtunkhwa,2.0,Chitral,206,Mastuj,20602.0,None,0,POINT (890277.037 4089274.401),29.0
24624,135201,5,10782803.0,11604427.0,36.874666,73.651880,36:52:29N,73:39:07E,43SCA7985081817,NJ43-14,...,Khyber Pakhtunkhwa,2.0,Chitral,206,Mastuj,20602.0,None,0,POINT (914714.445 4091088.767),0.0


### Filter down settlements

In [77]:
set_any = settle_pts_zs[settle_pts_zs['sum'] > 0]
set100 = settle_pts_zs[settle_pts_zs['sum'] > 100]
set200 = settle_pts_zs[settle_pts_zs['sum'] > 200]
set500 = settle_pts_zs[settle_pts_zs['sum'] > 500]
set1000 = settle_pts_zs[settle_pts_zs['sum'] > 1000]
# 1500 and above yield no points

In [78]:
print(len(set_any))
print(len(set100))
print(len(set200))
print(len(set500))
print(len(set1000))

10196
3863
2482
749
66


### Export

In [24]:
set200_zs.to_file(os.path.join(data_dir,vect_dir,"KP_Settlements_NGA_2017_200mBuff_ZS_Results_v2.gpkg"),\
                  layer='Buffered_Polygons',driver="GPKG")

# settle_pts_zs.to_file(os.path.join(data_dir,vect_dir,"KP_Settlements_NGA_2017_200mBuff_ZS_Results.gpkg"),\
#                       layer='Points',driver="GPKG")

In [25]:
set_any.to_file(os.path.join(data_dir,vect_dir,"KP_NGA_Settlements_200mBuff_Filtered.gpkg"),layer="Any",driver="GPKG")
set100.to_file(os.path.join(data_dir,vect_dir,"KP_NGA_Settlements_200mBuff_Filtered.gpkg"),layer="100plus",driver="GPKG")
set200.to_file(os.path.join(data_dir,vect_dir,"KP_NGA_Settlements_200mBuff_Filtered.gpkg"),layer="200plus",driver="GPKG")
set500.to_file(os.path.join(data_dir,vect_dir,"KP_NGA_Settlements_200mBuff_Filtered.gpkg"),layer="500plus",driver="GPKG")
set1000.to_file(os.path.join(data_dir,vect_dir,"KP_NGA_Settlements_200mBuff_Filtered.gpkg"),layer="1000plus",driver="GPKG")

## Develop settlement catchments and aggregate populations within them

We need to assign populations to settlements to capture their relative importance in the gravity model. This could be done with some sort of buffer but this is crude and would also create double counting issues when settlements are close, unless tedious and somewhat arbitrary difference calculations are imposed on the buffer polygons.</br></br>A better solution is to generate exclusive catchments for the settlements based on the friction surface itself, then summarize populations of interest within these catchments. This has the double benefit of avoiding double counting and also better reflecting how market structures actually work: farmers bring produce to the local town and sell it to middlemen who transport it onwards to the main markets.
</br></br>We use GOST's market catchment code in combination with our friction surface to prepare the catchments

Look at our settlements layer

In [79]:
set200.head()

,FID_Settle,RC,UFI,UNI,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,...,PROVINCE,PROVINCE_C,DISTRICT,DISTRICT_C,TEHSIL,TEHSIL_C,Remarks,hfs,geometry,sum
2,44285,5,6352950.0,6574703.0,31.327376,70.598907,31:19:39N,70:35:56E,42RXV5212766988,NH42-04,...,Khyber Pakhtunkhwa,2.0,Dera Ismail Khan,207,Dera Ismail Khan,20702.0,None,0,POINT (652125.948 3466991.740),230.0
6,44199,5,6140333.0,6574667.0,31.333203,70.663098,31:20:00N,70:39:47E,42RXV5822567724,NH42-04,...,Khyber Pakhtunkhwa,2.0,Dera Ismail Khan,207,Dera Ismail Khan,20702.0,None,0,POINT (658225.268 3467725.125),228.0
11,117250,5,357608.0,445255.0,31.346554,70.348931,31:20:48N,70:20:56E,42RXV2831568796,NH42-03,...,Khyber Pakhtunkhwa,4.0,FR Dera Ismail Khan,409,FR Dera Ismail Khan,40901.0,Pcode change,0,POINT (628311.787 3468790.647),465.0
13,44371,5,-2772314.0,-3849092.0,31.350885,70.687971,31:21:03N,70:41:17E,42RXV6056269721,NH42-04,...,Khyber Pakhtunkhwa,2.0,Dera Ismail Khan,207,Dera Ismail Khan,20702.0,None,0,POINT (660564.460 3469723.268),828.0
21,44218,5,6140331.0,6150497.0,31.385802,70.709125,31:23:09N,70:42:33E,42RXV6251473622,NH42-04,...,Khyber Pakhtunkhwa,2.0,Dera Ismail Khan,207,Dera Ismail Khan,20702.0,None,0,POINT (662511.697 3473623.051),243.0


In [80]:
set100.columns

Index(['FID_Settle', 'RC', 'UFI', 'UNI', 'LAT', 'LONG', 'DMS_LAT', 'DMS_LONG',
       'MGRS', 'JOG', 'FC', 'DSG', 'PC', 'CC1', 'ADM1', 'POP', 'ELEV', 'CC2',
       'NT', 'LC', 'SHORT_FORM', 'GENERIC', 'SORT_NAME_', 'FULL_NAME_',
       'FULL_NAME1', 'SORT_NAME1', 'FULL_NAM_1', 'FULL_NAM_2', 'NOTE',
       'MODIFY_DAT', 'DISPLAY', 'NAME_RANK', 'NAME_LINK', 'TRANSL_CD',
       'NM_MODIFY_', 'F_EFCTV_DT', 'F_TERM_DT', 'FID_Pak_ad', 'PROVINCE',
       'PROVINCE_C', 'DISTRICT', 'DISTRICT_C', 'TEHSIL', 'TEHSIL_C', 'Remarks',
       'hfs', 'geometry', 'sum'],
      dtype='object')

In [81]:
fric_dry = os.path.join(geo_pth,'Accessibility/KP_dry_friction1_210817.tif')

Create an mcp object from the friction surface raster

In [82]:
inR = rasterio.open(fric_dry)
inD = inR.read()[0,:,:] * inR.transform.a # important to specify pixel size in meters here in oder to get correct measurements
inD = np.nan_to_num(inD)
mcp = graph.MCP_Geometric(inD)

Find the MCP destination indices on the raster surface and stash them in a list of tuples

In [83]:
dests = ma.get_mcp_dests(inR, set100, makeset=False)

use the dests and mcp object to generate catchment areas around the settlements

In [84]:
ma.generate_market_sheds??

Signature:
ma.generate_market_sheds(
    inR,
    inH,
    out_file='',
    verbose=True,
    factor=1000,
    bandIdx=0,
)
Docstring:
identify pixel-level maps of market sheds based on travel time    
INPUTS
    inR [rasterio] - raster from which to grab index for calculations in MCP
    inH [geopandas data frame] - geopandas data frame of destinations
    factor [int] - value by which to multiply raster 
    
RETURNS
    [numpy array] - marketsheds by index
    
NOTES:
    Incredible help from StackOverflow:
    https://stackoverflow.com/questions/62135639/mcp-geometrics-for-calculating-marketsheds
    https://gist.github.com/bpstewar/9c15fc0948e82aa9667f1b04fd2c0295
Source:   
def generate_market_sheds(inR, inH, out_file='', verbose=True, factor=1000, bandIdx=0):
    ''' identify pixel-level maps of market sheds based on travel time    
    INPUTS
        inR [rasterio] - raster from which to grab index for calculations in MCP
        inH [geopandas data frame] - geopandas data fram

Generate the actual catchments -- this will take a while

In [86]:
set100.index

Int64Index([    2,     6,     8,    11,    13,    19,    21,    23,    25,
               27,
            ...
            24234, 24237, 24292, 24334, 24374, 24404, 24415, 24419, 24519,
            24551],
           dtype='int64', length=3863)

In [85]:
set100_catch = ma.generate_market_sheds(inR,set100,inR.transform.a)

ValueError: invalid entry in coordinates array

In [ ]:
with rasterio.open(geo_pth,'Accessibility/KP_Set200_Marketsheds_test.tif',**inR.profile) as dst:
    dst.write(set100_catch)

In [52]:
# set100_catch = ma.generate_feature_vectors(inR,mcp,set100,[30],'FID_Settle') # veeery slow

14:10:43	1 of 3863: 44285
14:30:14	2 of 3863: 44199


KeyboardInterrupt: 

### OLD

Join the catchments to the original dataset

In [ ]:
set100_catch.head()

In [ ]:
set100_catch_fin = gpd.GeoDataFrame(pd.merge(set100_catch,set100.drop('geometry',axis=1),how='left',on='FID_Settle'))

In [18]:
# run zonal stats on just the geometry column of set100, then join it back to the original GDF based on common indices

set100_catch_zs = set100.join(
    pd.DataFrame(
        zonal_stats(
                 vectors = set100_catch,\
                 raster = wp_kp,\
                 affine = wp_kp_raw.transform,\
                 stats = 'sum'
        )
    ),
    how='left',on='FID_Settle'
)

/Users/robert/.pyenv/versions/geo/lib/python3.9/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
